## データファイルの読み込み
- 遺伝子発現量・比較結果のファイル
- 遺伝子の詳細情報リスト（BLAST結果）
- 遺伝子のGO_IDリスト

In [ ]:
# ファイル名
RES_FILE = 'DE_result.txt'  # 遺伝子発現量・比較結果
BST_FILE = 'DEGs_BLAST.txt' # 詳細情報リスト（BLAST結果）
GO_FILE  = 'DEGs_GO.txt'    # GO_IDリスト

In [ ]:
# パッケージやツールをインポート
import pandas as pd
%matplotlib inline

# 読み込み
RES = pd.read_csv(RES_FILE, sep='\t', header=0,  index_col=0)
BST = pd.read_csv(BST_FILE, sep='\t', header=0,  index_col=0)
GO  = pd.read_csv(GO_FILE, sep='\t', header=-1, index_col=0, names=['GO'])

## 集計
- 遺伝子総数      : 調査した遺伝子数
- 発現変動遺伝子数 : 発現量に差があった遺伝子数（FDR<0.05）
- サンプルA高発現  : サンプルAで多く発現していた遺伝子数（FDR<0.05 & logFC>0）
- サンプルB高発現  : サンプルBで多く発現していた遺伝子数（FDR<0.05 & logFC<0）

In [ ]:
# 比較した遺伝子数
print('遺伝子総数:', len(RES))  

# 発現量に差があった遺伝子数
sub = RES[RES['FDR']<0.05]
print('発現変動遺伝子数:', len(sub))

# サンプルAで多く発現していた遺伝子数
subA = sub[sub['logFC']>0]
subB = sub[sub['logFC']<0]
print('サンプルA高発現:', len(subA))
print('サンプルB高発現:', len(subB))

# 棒グラフ
def bar_plot(df, log_scale=False):
    from matplotlib import pyplot as plt
    plt.style.use('ggplot')
    
    non = len(df) - len(df[df['FDR']<0.05])
    upA = len(df[(df['FDR']<0.05) & (df['logFC']>0)])
    upB = len(df[(df['FDR']<0.05) & (df['logFC']<0)])
    
    G = plt.bar(
        x      = ['High-A', 'High-B', 'non-DEGs'],
        height = [upA, upB, non], 
        align  = "center",
        log    = log_scale,
        facecolor='#9999ff',
        edgecolor='white',
    )
    
    # フォントサイズ
    plt.tick_params(labelsize=18)
    
    # カウント追加
    plt.text(0, upA*1.2, '%d'%upA, ha='center', va='bottom', size=18)
    plt.text(1, upB*1.2, '%d'%upB, ha='center', va='bottom', size=18)
    plt.text(2, non*0.9, '%d'%non, ha='center', va='bottom', size=18)
    # タイトル部分に遺伝子総数
    plt.title('Total: %d genes'%len(df), size=18)

bar_plot(RES)

## ワード検索

In [ ]:
# 検索ワード
KEYWORD = 'resistance'

# サンプルA高発現遺伝子のBLAST情報
subA_BST = pd.merge(subA, BST, left_index=True, right_index=True, how='left')
subA_BST[ subA_BST['sprot_Top_BLASTX_hit'].str.contains(KEYWORD) ]

In [ ]:
# サンプルB高発現遺伝子のBLAST情報
subB_BST = pd.merge(subB, BST, left_index=True, right_index=True, how='left')
subB_BST[ subB_BST['sprot_Top_BLASTX_hit'].str.contains(KEYWORD) ]

## 出現GO Term カウント

In [ ]:
# カウント関数
def count_go(df):
    import pandas as pd
    
    # All Go terms put into one list, "all_GO".
    # Some GO terms are duplicated in the "all_GO".
    all_GO = []
    for each_GOs in df["GO"]:
        GO_list = each_GOs.split(",")
        all_GO.extend(GO_list)

    # Counting GO terms using Pandas series
    result = pd.Series(all_GO).value_counts()

    return pd.DataFrame(result, columns=['count'])

# サンプルA高発現遺伝子のGO Termリスト
subA_GO = pd.merge(subA, GO, left_index=True, right_index=True, how='inner')
subB_GO = pd.merge(subB, GO, left_index=True, right_index=True, how='inner')

subA_CNT = count_go(subA_GO)
subA_CNT.columns = ['sampleA']
subB_CNT = count_go(subB_GO)
subB_CNT.columns = ['sampleB']

COUNT = pd.merge(subA_CNT, subB_CNT, left_index=True, right_index=True, how='outer').fillna(0)

# ファイル書き込み
COUNT.to_csv('DEGs_GO_count.txt', sep='\t', header=True, index=True)

# 表示（最初の10行）
COUNT.head(10)